In [3]:
import pandas as pd
import json
import os


In [4]:
# Definir la ruta base de los archivos
# Asegúrate de cambiar esta ruta si el dataset está en otra ubicación
RUTA_BASE = "C:/Users/oscar/Desktop/P1-HENRY/data_set"

In [5]:
# Cargar el archivo credits.csv
ruta_credits = os.path.join(RUTA_BASE, "credits.csv")
credits_df = pd.read_csv(ruta_credits, low_memory=False)


In [6]:
#  Ver las primeras filas
display(credits_df.head())

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [7]:
#  Convertir la columna 'id' a tipo entero
# Esto facilita la fusión que se hara más adelante con el dataset movie
credits_df['id'] = credits_df['id'].astype(int)

In [8]:
# Función para extraer los 5 actores principales
def extract_actors(json_str):
    """ Extrae los nombres de los 5 actores principales """
    if pd.isna(json_str):
        return None
    try:
# Convertir la cadena JSON en una lista de diccionarios            
        data = json.loads(json_str.replace("'", '"'))
        return ', '.join([item.get('name', '') for item in data[:5]]) if isinstance(data, list) else None
    except json.JSONDecodeError:
        return None
#Aplicar la función a la columna 'cast' para extraer los actores principales
credits_df['actors'] = credits_df['cast'].apply(extract_actors)


In [9]:
#  Función para extraer el director de la película
def extract_director(json_str):
    """ Extrae el nombre del director de la película """
    if pd.isna(json_str):
        return None
    try:
        data = json.loads(json_str.replace("'", '"'))
        for person in data:
            if person.get('job') == 'Director':
                return person.get('name', '')
    except json.JSONDecodeError:
        return None
    return None
# Aplicar la función a la columna 'crew' para extraer el director
credits_df['director'] = credits_df['crew'].apply(extract_director)

In [10]:
# Eliminar columnas innecesarias
# Se eliminan 'cast' y 'crew' ya que sus datos relevantes han sido extraídos
credits_df = credits_df.drop(columns=['cast', 'crew'])

In [11]:
# Guardar los datos transformados en formato Parquet para optimización
ruta_credits_parquet = os.path.join(RUTA_BASE, "credits.parquet")
credits_df.to_parquet(ruta_credits_parquet, index=False)

print(f" Transformación de `credits.csv` completada. Archivo guardado en:\n  {ruta_credits_parquet}")

 Transformación de `credits.csv` completada. Archivo guardado en:
  C:/Users/oscar/Desktop/P1-HENRY/data_set\credits.parquet
